In [77]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Lambda
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Softmax


In [7]:
with open('data/shakespeare_proc_rnn.txt', 'r') as f:
    source = f.read()

num_chars = len(set(source))
char_seqs = [source[i-40:i+1] for i in range(40, len(source))]
char_to_int = {}
for i, char in enumerate(set(source)):
    char_to_int[char] = i
encoded = np.array([[char_to_int[char] for char in seq] for seq in char_seqs])

In [8]:
X, y = encoded[:,:-1], encoded[:,-1]
X = np.array([to_categorical(x, num_classes=num_chars) for x in X])
y = to_categorical(y, num_classes=num_chars)
    
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(num_chars, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy')
model.fit(X, y, epochs=50)
model.save('rnn_model')

Epoch 1/50
93786/93786 [==============================] - 80s 857us/step - loss: 2.3191
Epoch 2/50
93786/93786 [==============================] - 79s 848us/step - loss: 1.9219
Epoch 3/50
93786/93786 [==============================] - 77s 817us/step - loss: 1.7793
Epoch 4/50
93786/93786 [==============================] - 79s 841us/step - loss: 1.6902
Epoch 5/50
93786/93786 [==============================] - 78s 827us/step - loss: 1.6235
Epoch 6/50
93786/93786 [==============================] - 81s 869us/step - loss: 1.5690
Epoch 7/50
93786/93786 [==============================] - 81s 861us/step - loss: 1.5207
Epoch 8/50
93786/93786 [==============================] - 77s 824us/step - loss: 1.4769
Epoch 9/50
93786/93786 [==============================] - 83s 882us/step - loss: 1.4353
Epoch 10/50
93786/93786 [==============================] - 78s 833us/step - loss: 1.3973
Epoch 11/50
93786/93786 [==============================] - 79s 844us/step - loss: 1.3606
Epoch 12/50
93786/93786 [=====

In [155]:
model = load_model('rnn_model')

temperature = 0.75

text = 'shall i compare thee to a summer\'s day?\n'
for i in range(582):
    encoded = [char_to_int[char] for char in text]
    encoded = pad_sequences([encoded], maxlen=40, truncating='pre')
    encoded = to_categorical(encoded, num_classes=len(char_to_int))
    preds = model.predict(encoded, verbose=0)
    pred = np.log(preds[0]) / temperature
    exp_pred = np.exp(pred)
    exp_pred /= (np.sum(exp_pred) + .00001)
    out_index = np.argmax(np.random.multinomial(1, exp_pred, 1))
    out_char = ''
    for char, index in char_to_int.items():
        if index == out_index:
            out_char = char
            break
    text += char

print(text)

shall i compare thee to a summer's day?
what this in their fair lend fair, cowers confound.
wherein thou wilt puse too, this in their skill,
to rides of seesured that her through my state,
and thou shoucds of thy hume, look doth gone,
savanieved do the could with the rearly year!
what offards do hours, or thy should thou grew?
o my love may state in summer's new earth,
far inmort noter, shall time thou wilt look, bein are metreace,
whereother-fair o'erity that your self is cride.
he piab a wond appear in race to reason,
presaded lipser thus thou not sore can sit,
thou condercess your his minure with their sorrow,
t
